# Imports

In [1]:
import logging

import pandas as pd

import helpers.hdatetime as hdateti
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hpandas as hpandas
import helpers.hprint as hprint
import im_v2.ccxt.data.extract.extractor as imvcdexex
import im_v2.common.data.client.im_raw_data_client as imvcdcimrdc
import im_v2.common.data.qa.dataset_validator as imvcdqdava
import im_v2.common.data.qa.qa_check as imvcdqqach
import im_v2.common.universe as ivcu
import im_v2.common.universe.universe as imvcounun

%load_ext autoreload
%autoreload 2

/app/core/signal_processing/decorrelation.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-29ea478c-d50f-4278-aab7-4bc577a73738.json'
-----------------------------------------------------------------------------
This code is not in sync with the container:
code_version='1.4.3' != container_version='1.4.0'
-----------------------------------------------------------------------------
You need to:
- merge origin/master into your branch with `invoke git_merge_master`
- pull the latest container with `invoke docker_pull`
INFO  # Git
  branch_name='CmTask4133_bigint_out_of_range_in_the_preprod_2'
  hash='b805d5864'
  # Last commits:
    * b805d5864 Grigorii Pomazkin Cm task4240 tweak history lookback (#4244)                        (    2 days ago) Mon Apr 17 18:27:45 2023  (HEAD -> CmTask4133_bigint_out_of_range_in_the_preprod_2, origin/master, origin/HEAD, master)
    * d34840d90 Daniil Tikhomirov CMTask4222: Add leverage to extra params (#4243)                  (    

In [3]:
# Imports

import logging

import pandas as pd

import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hpandas as hpandas
import helpers.hprint as hprint
import im_v2.ccxt.data.extract.extractor as imvcdexex
import im_v2.common.data.client.im_raw_data_client as imvcdcimrdc
import im_v2.common.data.qa.dataset_validator as imvcdqdava
import im_v2.common.data.qa.qa_check as imvcdqqach
import im_v2.common.universe as ivcu
import im_v2.common.universe.universe as imvcounun

%load_ext autoreload
%autoreload 2

hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
-----------------------------------------------------------------------------
This code is not in sync with the container:
code_version='1.4.3' != container_version='1.4.0'
-----------------------------------------------------------------------------
You need to:
- merge origin/master into your branch with `invoke git_merge_master`
- pull the latest container with `invoke docker_pull`
INFO  # Git
  branch_name='CmTask3979_Onboard_CCXT_Binance_futures_realtime_trades'
  hash='b4562aaa4'
  # Last commits:
    * b4562aaa4 Nina Lee Cm task3844 add unit tests for tca reader (#3984)                 (    3 days ago) Fri Mar 3 19:21:26 2023  (HEAD -> CmTask3979_Onboard_CCXT_Binance_futures_realtime_trades, origin/master, origin/HEAD, origin/CmTask3979_Onboard_CCXT_Binance_futures_realtime_trades, master)
    * 3f6320439 Nina Lee CMTask3944_fix_test_reconciliation_notebook (#3969)               (    3 days a

# Test _subscribe_to_websocket_trades

In [24]:
exchange_id = "binance"
contract_type = "futures"
extractor = imvcdexex.CcxtExtractor(exchange_id, contract_type)
extractor._async_exchange.has["watchTrades"]

True

In [25]:
currency_pair = "BTC/USDT"
since = hdateti.convert_timestamp_to_unix_epoch(
    pd.Timestamp.now() + pd.Timedelta(seconds=15)
)
await extractor._subscribe_to_websocket_bid_ask(
    exchange_id,
    currency_pair,
    bid_ask_depth=10
    #     since
)

## Test _download_websocket_trades(

In [26]:
extractor._download_websocket_bid_ask(
    exchange_id,
    currency_pair,
)

WARNING Timestamp is not an integer. currency_pair=BTC/USDT.


{'bids': [[29377.9, 0.182],
  [29377.8, 0.498],
  [29377.7, 0.108],
  [29377.6, 0.038],
  [29377.5, 0.001],
  [29377.4, 0.003],
  [29377.3, 0.021],
  [29377.2, 0.001],
  [29377.1, 0.003],
  [29377.0, 0.129]],
 'asks': [[29378.0, 32.113],
  [29378.1, 0.006],
  [29378.2, 0.512],
  [29378.3, 0.698],
  [29378.4, 0.016],
  [29378.5, 0.001],
  [29378.6, 0.001],
  [29378.7, 0.501],
  [29378.8, 0.546],
  [29378.9, 1.292]],
 'timestamp': 1681917874056,
 'datetime': '2023-04-19T15:24:34.056Z',
 'nonce': 2756065658099,
 'symbol': 'BTC/USDT',
 'end_download_timestamp': '2023-04-19 15:24:34.230564+00:00'}

# Download by script

In [28]:
%%bash
/app/im_v2/ccxt/data/extract/download_exchange_data_to_db_periodically.py \
    --download_mode 'realtime' \
    --downloading_entity 'manual' \
    --action_tag 'downloaded_1sec' \
    --vendor 'ccxt' \
    --exchange_id 'binance' \
    --universe 'v7.3' \
    --db_stage 'test' \
    --db_table 'ccxt_bid_ask_futures_raw' \
    --aws_profile 'ck' \
    --data_type 'bid_ask' \
    --data_format 'postgres' \
    --contract_type 'futures' \
    --start_time "$(date -d 'today + 1 minutes' +'%Y-%m-%d %H:%M:00+00:00')" \
    --stop_time "$(date -d 'today + 3 minutes' +'%Y-%m-%d %H:%M:00+00:00')" \
    --method 'websocket'


INFO: > cmd='/app/im_v2/ccxt/data/extract/download_exchange_data_to_db_periodically.py --download_mode realtime --downloading_entity manual --action_tag downloaded_1sec --vendor ccxt --exchange_id binance --universe v7.3 --db_stage test --db_table ccxt_bid_ask_futures_raw --aws_profile ck --data_type bid_ask --data_format postgres --contract_type futures --start_time 2023-04-19 15:37:00+00:00 --stop_time 2023-04-19 15:39:00+00:00 --method websocket'
report_memory_usage=False report_cpu_usage=False
INFO: Saving log to file '/app/im_v2/ccxt/data/extract/download_exchange_data_to_db_periodically.py.log'
11:36:30 - INFO  Task-1 db_utils.py get_connection:85                   Unable to fetch DB credentials from environment variables: 
	'POSTGRES_HOST'
	Attempting env file method.
11:36:30 - INFO  Task-1 db_utils.py get_connection:106                  Created test DB connection: 
 <connection object at 0x7f2601d4e7c0; dsn: 'user=ck_test password=xxx dbname=test.im_data_db host=dev-im-db.cpox

11:37:46 - INFO  Task-1 extract_utils.py _download_websocket_realtime_for_one_exchange_periodically:442 Iteration took 261 ms, waiting between iterations for 0 ms
11:37:46 - INFO  Task-1 extract_utils.py _download_websocket_realtime_for_one_exchange_periodically:442 Iteration took 117 ms, waiting between iterations for 82 ms
11:37:47 - INFO  Task-1 extract_utils.py _download_websocket_realtime_for_one_exchange_periodically:442 Iteration took 275 ms, waiting between iterations for 0 ms
11:37:47 - INFO  Task-1 extract_utils.py _download_websocket_realtime_for_one_exchange_periodically:442 Iteration took 117 ms, waiting between iterations for 82 ms
11:37:47 - INFO  Task-1 extract_utils.py _download_websocket_realtime_for_one_exchange_periodically:442 Iteration took 300 ms, waiting between iterations for 0 ms
11:37:47 - INFO  Task-1 extract_utils.py _download_websocket_realtime_for_one_exchange_periodically:442 Iteration took 120 ms, waiting between iterations for 79 ms
11:37:48 - INFO  Ta

11:38:39 - INFO  Task-1 extract_utils.py _download_websocket_realtime_for_one_exchange_periodically:442 Iteration took 119 ms, waiting between iterations for 80 ms
11:38:49 - INFO  Task-1 extract_utils.py _download_websocket_realtime_for_one_exchange_periodically:442 Iteration took 9486 ms, waiting between iterations for 0 ms
11:38:49 - INFO  Task-1 extract_utils.py _download_websocket_realtime_for_one_exchange_periodically:442 Iteration took 120 ms, waiting between iterations for 79 ms
11:38:49 - INFO  Task-1 extract_utils.py _download_websocket_realtime_for_one_exchange_periodically:442 Iteration took 278 ms, waiting between iterations for 0 ms
11:38:50 - INFO  Task-1 extract_utils.py _download_websocket_realtime_for_one_exchange_periodically:442 Iteration took 117 ms, waiting between iterations for 82 ms
11:38:50 - INFO  Task-1 extract_utils.py _download_websocket_realtime_for_one_exchange_periodically:442 Iteration took 284 ms, waiting between iterations for 0 ms
11:38:50 - INFO  T

## Check by reader

In [29]:
signature = "realtime.airflow.downloaded_1sec.postgres.bid_ask.futures.v7_3.ccxt.binance.v1_0_0"
reader = imvcdcimrdc.RawDataReader(signature, stage="test")
start_timestamp = pd.Timestamp("2023-04-19 00:40:00+00:00")
end_timestamp = pd.Timestamp("2023-04-19 23:40:00+00:00")

INFO  Loading dataset schema file: /app/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3
INFO  Loading dataset schema file: /app/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3
INFO  Unable to fetch DB credentials from environment variables: 
	'POSTGRES_HOST'
	Attempting env file method.
INFO  Created test DB connection: 
 <connection object at 0x7f9314b00a40; dsn: 'user=ck_test password=xxx dbname=test.im_data_db host=dev-im-db.cpox8ul7pzan.eu-north-1.rds.amazonaws.com port=5432', closed: 0>
INFO  Enabled connection to the `ccxt_bid_ask_futures_raw` DB table


In [30]:
trades = reader.read_data(start_timestamp, end_timestamp)

INFO  Executing query: 
	SELECT * FROM ccxt_bid_ask_futures_raw WHERE timestamp >= 1681864800000 AND timestamp <= 1681947600000


/app/helpers/hsql_implementation.py:582: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


In [31]:
trades

,currency_pair,exchange_id,end_download_timestamp,knowledge_timestamp,bid_size_l1,bid_size_l2,bid_size_l3,bid_size_l4,bid_size_l5,bid_size_l6,bid_size_l7,bid_size_l8,bid_size_l9,bid_size_l10,bid_size_l11,bid_size_l12,bid_size_l13,bid_size_l14,bid_size_l15,bid_size_l16,bid_size_l17,bid_size_l18,bid_size_l19,bid_size_l20,bid_size_l21,bid_size_l22,bid_size_l23,bid_size_l24,bid_size_l25,bid_size_l26,bid_size_l27,bid_size_l28,bid_size_l29,bid_size_l30,bid_size_l31,bid_size_l32,bid_size_l33,bid_size_l34,bid_size_l35,bid_size_l36,bid_size_l37,bid_size_l38,bid_size_l39,bid_size_l40,bid_size_l41,bid_size_l42,bid_size_l43,bid_size_l44,bid_size_l45,bid_size_l46,bid_size_l47,bid_size_l48,bid_size_l49,bid_size_l50,bid_size_l51,bid_size_l52,bid_size_l53,bid_size_l54,bid_size_l55,bid_size_l56,bid_size_l57,bid_size_l58,bid_size_l59,bid_size_l60,bid_size_l61,bid_size_l62,bid_size_l63,bid_size_l64,bid_size_l65,bid_size_l66,bid_size_l67,bid_size_l68,bid_size_l69,bid_size_l70,bid_size_l71,bid_size_l72,bid_size_l73,bid_size_l74,bid_size_l75,bid_size_l76,bid_size_l77,bid_size_l78,bid_size_l79,bid_size_l80,bid_size_l81,bid_size_l82,bid_size_l83,bid_size_l84,bid_size_l85,bid_size_l86,bid_size_l87,bid_size_l88,bid_size_l89,bid_size_l90,bid_size_l91,bid_size_l92,bid_size_l93,bid_size_l94,bid_size_l95,bid_size_l96,bid_size_l97,bid_size_l98,bid_size_l99,bid_size_l100,bid_size_l101,bid_size_l102,bid_size_l103,bid_size_l104,bid_size_l105,bid_size_l106,bid_size_l107,bid_size_l108,bid_size_l109,bid_size_l110,bid_size_l111,bid_size_l112,bid_size_l113,bid_size_l114,bid_size_l115,bid_size_l116,bid_size_l117,bid_size_l118,bid_size_l119,bid_size_l120,bid_size_l121,bid_size_l122,bid_size_l123,bid_size_l124,bid_size_l125,bid_size_l126,bid_size_l127,bid_size_l128,bid_size_l129,bid_size_l130,bid_size_l131,bid_size_l132,bid_size_l133,bid_size_l134,bid_size_l135,bid_size_l136,bid_size_l137,bid_size_l138,bid_size_l139,bid_size_l140,bid_size_l141,bid_size_l142,bid_size_l143,bid_size_l144,bid_size_l145,bid_size_l146,bid_size_l147,bid_size_l148,bid_size_l149,bid_size_l150,bid_size_l151,bid_size_l152,bid_size_l153,bid_size_l154,bid_size_l155,bid_size_l156,bid_size_l157,bid_size_l158,bid_size_l159,bid_size_l160,bid_size_l161,bid_size_l162,bid_size_l163,bid_size_l164,bid_size_l165,bid_size_l166,bid_size_l167,bid_size_l168,bid_size_l169,bid_size_l170,bid_size_l171,bid_size_l172,bid_size_l173,bid_size_l174,bid_size_l175,bid_size_l176,bid_size_l177,bid_size_l178,bid_size_l179,bid_size_l180,bid_size_l181,bid_size_l182,bid_size_l183,bid_size_l184,bid_size_l185,bid_size_l186,bid_size_l187,bid_size_l188,bid_size_l189,bid_size_l190,bid_size_l191,bid_size_l192,bid_size_l193,bid_size_l194,bid_size_l195,bid_size_l196,bid_size_l197,bid_size_l198,bid_size_l199,bid_size_l200,bid_size_l201,bid_size_l202,bid_size_l203,bid_size_l204,bid_size_l205,bid_size_l206,bid_size_l207,bid_size_l208,bid_size_l209,bid_size_l210,bid_size_l211,bid_size_l212,bid_size_l213,bid_size_l214,bid_size_l215,bid_size_l216,bid_size_l217,bid_size_l218,bid_size_l219,bid_size_l220,bid_size_l221,bid_size_l222,bid_size_l223,bid_size_l224,bid_size_l225,bid_size_l226,bid_size_l227,bid_size_l228,bid_size_l229,bid_size_l230,bid_size_l231,bid_size_l232,bid_size_l233,bid_size_l234,bid_size_l235,bid_size_l236,bid_size_l237,bid_size_l238,bid_size_l239,bid_size_l240,bid_size_l241,bid_size_l242,bid_size_l243,bid_size_l244,bid_size_l245,bid_size_l246,...,ask_price_l751,ask_price_l752,ask_price_l753,ask_price_l754,ask_price_l755,ask_price_l756,ask_price_l757,ask_price_l758,ask_price_l759,ask_price_l760,ask_price_l761,ask_price_l762,ask_price_l763,ask_price_l764,ask_price_l765,ask_price_l766,ask_price_l767,ask_price_l768,ask_price_l769,ask_price_l770,ask_price_l771,ask_price_l772,ask_price_l773,ask_price_l774,ask_price_l775,ask_price_l776,ask_price_l777,ask_price_l778,ask_price_l779,ask_price_l780,ask_price_l781,ask_price_l782,ask_price_l783,ask_price_l784,ask_price_l785,ask_price_l786,ask_price_l787,ask_price_l788,ask_price_l789,ask_p

In [32]:
trades.timestamp.agg(["min", "max"]).apply(pd.Timestamp, unit="ms")

WARNING binance requires to release all resources with an explicit call to the .close() coroutine. If you are using the exchange instance with async coroutines, add `await exchange.close()` to your code into a place when you're done with the exchange and don't need the exchange instance anymore (at the end of your async coroutine).
WARNING binance requires to release all resources with an explicit call to the .close() coroutine. If you are using the exchange instance with async coroutines, add `await exchange.close()` to your code into a place when you're done with the exchange and don't need the exchange instance anymore (at the end of your async coroutine).
WARNING binance requires to release all resources with an explicit call to the .close() coroutine. If you are using the exchange instance with async coroutines, add `await exchange.close()` to your code into a place when you're done with the exchange and don't need the exchange instance anymore (at the end of your async coroutine)

AttributeError: 'DataFrame' object has no attribute 'timestamp'